## Connecting to Redis
To connect to a single Redis instance:

In [ ]:
Config config = new Config();
config.useSingleServer()
        .setAddress("redis://127.0.0.1:6379");

RedissonClient client = Redisson.create(config);

To connect to a Redis cluster:

In [ ]:
config.useClusterServers()
        .addNodeAddress("redis://127.0.0.1:6379");

## Distributed Data Collection
<div style="display:inline-block" />

| Redis Type | Redisson Equivalent   | Extends       |
|------------|-----------------------|---------------|
| String     | `RBucket`             |               |
| List       | `RList`               | `List`        |
| Set        | `RSet`                | `Set`         |
| Hash       | `RMap`                | `ConcurretMap`|
| Sorted Set | `RScoredSortedSet`    |               |
</div>

In [ ]:
// String
RBucket<String> bucket = client.getBucket("counter");
bucket.set("100");
System.out.println(bucket.get());

// List
RList<String> list = client.getList("cities:africa");
list.add("Dakar");
list.add("Rabat");
list.add("Pikine");
System.out.println(list.size());

// Set
RSet<String> set = client.getSet("alphabets");
set.add("A");
set.add("B");
set.add("A");
System.out.println(set.contains("B"));

// Hash
RMap<String, Integer> map = client.getMap("car:veyron");
map.put("cylinders", 16);
map.put("power", 1001);
System.out.println(map.get("cylinders"));

// Sorted Set
RScoredSortedSet<String> scoredSet = client.getScoredSortedSet("scores");
scoredSet.add(9.5, "Alice");
scoredSet.add(8.0, "Bob");
System.out.println(scoredSet.first());

All the above data collections are thread safe stateless proxies. They do not store the data within the Java object itself; instead, they translate method calls (like `add()`, `put()`, or `get()`) into Redis commands. This means call to `get()` for example would get the latest data as it is on the server.

### String
`RBucket` is generic container for singular scalar values. Redisson provides atomic scalar types that are flavours of `RBucket`. `RAtomicLong` and `RAtomicDouble` act upon keys that have numeric content.

In [ ]:
RAtomicLong counter = client.getAtomicLong("counter");
counter.setIfGreater(100, 99);
System.out.println(counter.get());

These collections are used for single values where we need atomic updates across multiple servers.

### List
Other than `RList`, Redisson provides other interfaces like `RDeque`, `RQueue` and `RBlockingQueue`.
<div style="display:inline-block" />
    
| Interface      | Extends               | Implementation        |
|----------------|-----------------------|-----------------------|
| RDeque         | Queue, RQueue         | RedissonDeque         |
| RQueue         | Queue                 | RedissonQueue         |
| RBlockingQueue | BlockingQueue, RQueue | RedissonBlockingQueue |
</div>

Example usage:

In [ ]:
RBlockingQueue<String> videoTasks = client.getBlockingQueue("video:transcode:tasks");

// Worker Thread Logic:
while (true) {
    // This line blocks the thread until a task arrives
    String chunkId = videoTasks.take(); 
    processVideo(chunkId);
}

### Set and Sorted Set
Other than `RSet` and `RScoredSortedSet` shown earlier there are other implementations like `RSortedSet` and `RLexSortedSet`.

In [ ]:
// Sorted alphabetically by default, it is wierd one because it is stored as List in Redis
RSortedSet<String> directory = client.getSortedSet("company:directory");

directory.add("Zebra");
directory.add("Apple");
directory.add("Mango");

`RLexSortedSet` is saved as ZSET. Redisson assigns 0 score to each thus Redis sorts values lexicographically.

### Map
In addition to `RMap`, Redisson provides multimap implementations `RSetMultimap` and `RListMultimap`

In [ ]:
RSetMultimap<String, String> userRoles = client.getSetMultimap("system:user:roles");
userRoles.put("user_123", "ADMIN");
userRoles.put("user_123", "EDITOR");
userRoles.put("user_123", "ADMIN");
System.out.println(userRoles.containsEntry("user_123", "ADMIN"));

Map provides several atomic operations:
- `putIfAbsent(K key, V value)`: only inserts if the key doesn't exist.
- `replace(K key, V value)`: only updates if the key already exists.
- `fastPut(K key, V value)`: adds the data but doesn't return the previous value (saves network bandwidth).
- `addAndGet(K key, Number delta)`: atomically increments a numeric field inside the map (very common for analytics).
- `remove(Object key, Object value)`: removes only if the key is currently mapped to a specific value.
- `replace(K key, V oldValue, V newValue)`: standard CAS (Compare-And-Swap) logic.

## Distributed Locking
Redisson provides an distributed implementation of `Lock` called `RLock`. It ensures:
- Only one thread across all nodes holds the lock at a time
- Lock ownership is reentrant (same thread can re-lock)
- Locks are automatically released if the process dies

When a thread acquires an `RLock`, Redisson sets a "lease time" (default 30 seconds). A background task called *watchdog* checks the lock every 10 seconds (leaseTime / 3). If the thread is still alive and holding the lock, the watchdog automatically extends the expiration back to 30 seconds. If the server crashes, the watchdog dies with it. Redis will then naturally expire the lock after the remaining lease time, allowing other servers to claim it.

Example usage:

In [ ]:
RLock lock = redisson.getLock("payment:order:99");

// waitTime: 10s (Wait up to 10s if someone else has it)
// leaseTime: 30s (Lock will auto-release after 30s if app crashes)
if (lock.tryLock(10, 30, TimeUnit.SECONDS)) {
    try {
        processPayment();
    } finally {
        lock.unlock();
    }
} else {
    // couldn't acquire lock in 5 seconds
}

Some other available options:
<div style="display:inline-block">

|                  | Use Case                                                                                                         |
|------------------|------------------------------------------------------------------------------------------------------------------|
| `RFairLock`      | When you need a strict First-In-First-Out (FIFO) order for lock acquisition.                                     |
| `RReadWriteLock` | Allows multiple "Readers" to access a resource at once, but only one "Writer" (standard Read-Write pattern).     |
| `RMultiLock`     | Allows you to group multiple locks and treat them as one. You only get the lock if you acquire all of them.      |
| `RSemaphore`     | Restricts access to a resource to a specific number of concurrent threads (e.g., limiting database connections). |
</div>

Example usage of `RReadWriteLock`:

In [ ]:
RMap<String, Integer> inventory = client.getMap("inventory");
RReadWriteLock rwLock = client.getReadWriteLock("lock:inventory");

// Writer node
RLock writeLock = rwLock.writeLock();
writeLock.lock();

try {
    Integer currentStock = inventory.get(productId);
    if (currentStock == null || currentStock < quantity) {
        return false; // not enough stock
    }

    int newStock = currentStock - quantity;
    inventory.put(productId, newStock);

    System.out.println("Stock updated by " + quantity + ", remaining = " + newStock);
    return true;
} finally {
    writeLock.unlock();
}

// Reader node
RLock readLock = rwLock.readLock();
readLock.lock();
    
try {
    Integer stock = inventory.get(productId);
    return stock != null ? stock : 0;
} finally {
    readLock.unlock();
}